# Trabalho Prático 3 de Estruturas Criptográficas

- **Autores:** (Grupo 9)
     - Nelson Faria (A84727)
     - Miguel Oliveira (A83819)

## *Esquemas pós-quânticos de assinaturas digitais* - CRYSTALS-Dilithium

Nota: Baseado no documento CRYSTALS-Dilithium (https://pq-crystals.org/dilithium/data/dilithium-specification-round3-20210208.pdf)
